In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
def load_csvs(folder):
    data_path = f"C:\\path\\to\\expts\\{folder}"
    focal_L1 = pd.read_csv(f"{data_path}\\contrast_validation_focal_L1.csv", index_col=0, header=[0, 1])
    global_L1 = pd.read_csv(f"{data_path}\\contrast_validation_global.csv", index_col=0, header=[0, 1])

    if folder != "multi":
        focal_L1 = focal_L1[[c for c in focal_L1.columns if len(c[1].split('-')) < 3]]
        global_L1 = global_L1[[c for c in global_L1.columns if len(c[1].split('-')) < 3]]

    focal_L1.columns = pd.MultiIndex.from_tuples([(c[0], c[1]) for c in focal_L1.columns])
    global_L1.columns = pd.MultiIndex.from_tuples([(c[0], c[1]) for c in global_L1.columns])

    return focal_L1, global_L1

In [ ]:
def merge_kidney_rois(df):
    right_kidney = df["RK"]
    no_left_kidney = df["LK"].isna().any(axis=1)
    right_kidney.loc[~no_left_kidney] = (right_kidney.loc[~no_left_kidney] + df.loc[~no_left_kidney, "LK"]) / 2
    right_kidney.columns = pd.MultiIndex.from_product([["KD"], right_kidney.columns])
    df = pd.concat([df, right_kidney], axis=1)
    df.drop(["RK", "LK", "VC"], axis=1, inplace=True)
    return df

In [ ]:
ce_cols = [f"ce_vq{2**i}" for i in range(11)]
jo_cols = [f"jo_vq{2**i}" for i in range(11)]

def reorder_dfs(focal_df, global_df):
    focal_df = focal_df.loc[:, pd.IndexSlice[:, ["ce_vq0"] + ce_cols + jo_cols]]
    global_df = global_df.loc[:, pd.IndexSlice["L1", ["ce_vq0"] + ce_cols + jo_cols]]
    return focal_df, global_df

In [ ]:
focal_deep, global_deep = load_csvs("deep")
focal_deep = merge_kidney_rois(focal_deep)
focal_deep, global_deep = reorder_dfs(focal_deep, global_deep)

focal_shallow, global_shallow = load_csvs("shallow")
focal_shallow = merge_kidney_rois(focal_shallow)
focal_shallow, global_shallow = reorder_dfs(focal_shallow, global_shallow)

focal_multi, global_multi = load_csvs("multi")
focal_multi = merge_kidney_rois(focal_multi)
global_multi = global_multi.loc[:, pd.IndexSlice["L1", :]]
global_multi.rename({"ce_vqda1": "ce_vq128-128-512-1024-64", "jo_vqda1": "jo_vq128-128-256-512-512"}, axis=1, level=1, inplace=True)
focal_multi.rename({"ce_vqda1": "ce_vq128-128-512-1024-64", "jo_vqda1": "jo_vq128-128-256-512-512"}, axis=1, level=1, inplace=True)

In [ ]:
idx = global_deep.columns.get_level_values(1).unique()
pd.concat([global_deep.median().unstack().T, focal_deep.median().unstack().T], axis=1).loc[idx]

In [ ]:
idx = global_shallow.columns.get_level_values(1).unique()
pd.concat([global_shallow.median().unstack().T, focal_shallow.median().unstack().T], axis=1).loc[idx]  # ce_vq8

In [ ]:
pd.concat([global_multi.median().unstack().T, focal_multi.median().unstack().T], axis=1)#.loc[idx]

In [ ]:
def plot_boxplots(global_df, focal_df):
    plt.figure(figsize=(18, 18))
    plt.subplot(3, 1, 1)
    plt.boxplot(global_df["L1"])
    plt.xticks(ticks=np.arange(1, global_df["L1"].shape[1] + 1), labels=global_df["L1"].columns, rotation="vertical")
    plt.subplot(3, 1, 2)
    plt.boxplot(focal_df["AO"])
    plt.xticks(ticks=np.arange(1, focal_df["AO"].shape[1] + 1), labels=focal_df["AO"].columns, rotation="vertical")
    plt.subplot(3, 1, 3)
    plt.boxplot(focal_df["KD"])
    plt.xticks(ticks=np.arange(1, focal_df["KD"].shape[1] + 1), labels=focal_df["KD"].columns, rotation="vertical")
    plt.show()


In [ ]:
def calc_kruskal(global_df, focal_df):
    print(f"Global: {stats.kruskal(*global_df['L1'].values.T.tolist())}")
    print(f"Focal AO: {stats.kruskal(*focal_df['AO'].values.T.tolist())}")
    print(f"Focal KD: {stats.kruskal(*focal_df['KD'].values.T.tolist())}")

In [ ]:
g_df = global_shallow
f_df = focal_shallow
df_ce_cols = [c for c in g_df["L1"].columns if "ce" in c and c != "ce_vq0"]
df_jo_cols = [c for c in g_df["L1"].columns if "jo" in c]

plot_boxplots(g_df.loc[:, pd.IndexSlice[:, df_ce_cols]], f_df.loc[:, pd.IndexSlice[:, df_ce_cols]])
calc_kruskal(g_df.loc[:, pd.IndexSlice[:, df_ce_cols]], f_df.loc[:, pd.IndexSlice[:, df_ce_cols]])